# Estimating Coordinated Daily Activity Patterns

This notebook illustrates how to re-estimate ActivitySim's CDAP model. This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import pandas as pd
import larch as lx
import os

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare, backup

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

In [3]:
backup("output-est-mode/estimation_data_bundle/cdap/cdap_INDIV_AND_HHSIZE1_SPEC.csv")


# Load data and prep model for estimation

In [4]:
from activitysim.estimation.larch import component_model

modelname = "cdap"

model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

Reading output-est-mode/estimation_data_bundle/cdap/cdap_model_settings.yaml
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimation-data/activitysim-prototype-mtc-extended/output-est-mode/final_households.csv
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimation-data/activitysim-prototype-mtc-extended/output-est-mode/final_persons.csv
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimation-data/activitysim-prototype-mtc-extended/output-est-mode/estimation_data_bundle/cdap/cdap_coefficients.csv
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimation-data/activitysim-prototype-mtc-extended/output-est-mode/estimation_data_bundle/cdap/cdap_interaction_coefficients.csv
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimatio

# Review Inputs

## Settings

In [5]:
data.settings

┣          ADD_JOINT_TOUR_UTILITY: False
┣                    COEFFICIENTS: 'cdap_coefficients.csv'
┣                       CONSTANTS: ┣    DRIVING: 6
┃                                  ┣       FULL: 1
┃                                  ┣    NONWORK: 4
┃                                  ┣       PART: 2
┃                                  ┣  PRESCHOOL: 8
┃                                  ┣    RETIRED: 5
┃                                  ┣     SCHOOL: 7
┃                                  ┣ UNIVERSITY: 3
┣ FIXED_RELATIVE_PROPORTIONS_SPEC: 'cdap_fixed_relative_proportions.csv'
┣          INDIV_AND_HHSIZE1_SPEC: 'cdap_indiv_and_hhsize1.csv'
┣        INTERACTION_COEFFICIENTS: 'cdap_interaction_coefficients.csv'
┣         JOINT_TOUR_COEFFICIENTS: 'cdap_joint_tour_coefficients.csv'
┣       JOINT_TOUR_USEFUL_COLUMNS: None
┣                 PERSON_TYPE_MAP: ┣  CHILD: ---
┃                                  ┃         - 6
┃                                  ┃         - 7
┃                          

## Households

In [6]:
data.households

,home_zone_id,income,hhsize,HHT,auto_ownership,num_workers,sample_rate,income_in_thousands,income_segment,median_value_of_time,...,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,household_id
0,52,0,1,4,1,0,1,0.0,1,6.01,...,0,1,1,0,0,1,False,0_tours,0,45
1,124,14000,1,4,1,0,1,14.0,1,6.01,...,0,1,1,0,0,1,False,0_tours,0,499
2,132,9600,1,4,1,0,1,9.6,1,6.01,...,0,1,1,0,0,1,False,0_tours,0,659
3,148,18000,1,6,1,0,1,18.0,1,6.01,...,0,1,1,0,0,1,False,0_tours,0,948
4,166,14000,1,4,1,0,1,14.0,1,6.01,...,0,1,1,0,0,1,False,0_tours,0,1276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1386,0,1,0,1,0,1,0.0,1,6.01,...,0,1,1,0,0,1,False,0_tours,0,2874468
19996,1402,0,1,0,0,0,1,0.0,1,6.01,...,0,0,0,0,0,0,False,0_tours,0,2874567
19997,1402,0,1,0,1,0,1,0.0,1,6.01,...,0,1,1,0,0,1,False,0_tours,0,2874576
19998,1421,0,1,0,1,0,1,0.0,1,6.01,...,0,1,1,0,0,1,False,0_tours,0,2874826


## Person Data

In [7]:
data.person_data

,person_id,model_choice,override_choice,household_id,age,PNUM,sex,pemploy,pstudent,ptype,...,auOpRetail,auOpTotal,trPkRetail,trPkTotal,trOpRetail,trOpTotal,nmRetail,nmTotal,chunk_id,cdap_rank
0,45,N,N,45,48,1,1,3,3,4,...,9.924660,12.562639,4.193237,6.875144,3.952128,6.590585,2.194792,6.359507,0,1
1,499,N,N,499,50,1,1,3,3,4,...,10.284673,12.868645,6.639963,9.364105,6.531079,9.259002,5.955868,7.795004,1,1
2,659,N,N,659,52,1,1,3,3,4,...,10.247505,12.762286,6.001466,8.409169,5.786652,8.279842,5.798886,7.900061,0,1
3,948,N,N,948,61,1,2,3,3,4,...,10.150335,12.777635,5.172974,7.850360,4.893929,7.571579,4.895220,7.409345,1,1
4,1276,H,N,1276,46,1,1,3,3,4,...,10.258471,12.759529,6.039019,8.348963,5.778785,8.070525,6.073537,7.851667,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53069,7565803,N,N,2874468,85,1,2,3,3,5,...,8.113608,10.265845,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9998,1
53070,7565902,H,H,2874567,87,1,2,3,3,5,...,6.560015,8.886403,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9997,1
53071,7565911,N,N,2874576,85,1,2,3,3,5,...,6.560015,8.886403,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9999,1
53072,7566161,N,N,2874826,79,1,1,3,3,5,...,9.298380,11.721935,1.052528,2.925968,0.494776,2.006432,3.782008,6.208875,9998,1


## interaction_coef

In [8]:
data.interaction_coef

,activity,interaction_ptypes,coefficient,cardinality,slug
0,H,11,1.6260,2,HH11
1,H,12,0.7407,2,HH12
2,H,13,1.1830,2,HH13
3,H,14,0.9436,2,HH14
4,H,15,1.2980,2,HH15
...,...,...,...,...,...
128,N,,-1.3460,4,NNNN
129,H,,-3.7330,4,HHHH
130,M,,-1.5280,5,MMMMM
131,N,,-3.4530,5,NNNNN


## Spec 1

In [9]:
data.spec1.head()

,Description,Expression,M,N,H
0,Full-time worker alternative-specific constants,ptype == 1,coef_full_time_worker_asc_M,coef_full_time_worker_asc_N,NaN
1,Part-time worker alternative-specific constants,ptype == 2,coef_part_time_worker_asc_M,coef_part_time_worker_asc_N,NaN
2,University student alternative-specific constants,ptype == 3,coef_university_student_asc_M,coef_university_student_asc_N,NaN
3,Non-working adult alternative-specific constants,ptype == 4,coef_UNAVAILABLE,coef_non_working_adult_asc_N,NaN
4,Retired alternative-specific constants,ptype == 5,coef_UNAVAILABLE,coef_retired_asc_N,NaN


In [10]:
data.keys()

dict_keys(['edb_directory', 'person_data', 'spec1', 'interaction_coef', 'coefficients', 'households', 'settings', 'joint_coef', 'add_joint'])

# Review Model

## Utility Functions for HHSize 1

In [11]:
model[0].utility_functions() ## HH Size = 1 

<xmle.Elem 'div' with 1 children>

## Utility Functions for HHSize 2

In [12]:
model[1].utility_functions() ## HH Size = 2

<xmle.Elem 'div' with 1 children>

# Re-Estimate Model

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has two built-in estimation methods: BHHH and SLSQP.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.  SLSQP is safer, but slower, and may need additional iterations.

In [13]:
r = model.estimate(method='SLSQP', options={'maxiter':1000})

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
-999.0,-999.000000,-999.000000,-999.000000,-999.0,-999.0,0.0,1
coef_H_11,1.446554,1.446554,1.626000,-inf,inf,0.0,0
coef_H_12,0.954554,0.954554,0.740700,-inf,inf,0.0,0
coef_H_124_122_144,0.353981,0.353981,0.957300,-inf,inf,0.0,0
coef_H_126_146,-0.013374,-0.013374,0.293900,-inf,inf,0.0,0
...,...,...,...,...,...,...,...
coef_retired_interaction_with_more_cars_than_workers_M,2.992000,2.992000,2.992000,-inf,inf,0.0,0
coef_retired_interaction_with_more_cars_than_workers_N,1.015905,1.015905,1.056000,-inf,inf,0.0,0
coef_retired_interaction_with_peak_accessibility_to_all_employment_M,0.279200,0.279200,0.279200,-inf,inf,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/optimization.py:338: UserWarning: SLSQP may not play nicely with unbounded parameters
if you get poor results, consider setting global bounds with model.set_cap()
  warnings.warn(  # infinite bounds #  )


In [14]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
Parameter,,,,,
-999.0,-999.,0.00,NA,,0.00
coef_H_11,1.45,0.0614,23.55,***,0.00
coef_H_12,0.955,0.132,7.21,***,0.00
coef_H_124_122_144,0.354,0.153,2.31,*,0.00
coef_H_126_146,-0.0134,0.336,-0.04,,0.00
coef_H_13,1.20,0.130,9.21,***,0.00
coef_H_14,0.998,0.0903,11.05,***,0.00
coef_H_15,1.18,0.0922,12.76,***,0.00
coef_H_16,1.85,0.102,18.08,***,0.00


# Output Estimation Results

In [15]:
from activitysim.estimation.larch import update_coefficients
coefficients = update_coefficients(model, data)

In [16]:
result_dir = data.edb_directory/'estimated'

In [17]:
os.makedirs(result_dir, exist_ok=True)

### Write the re-estimated coefficients file

In [18]:
coefficients.reset_index().to_csv(
    result_dir/"cdap_coefficients_revised.csv", 
    index=False,
)

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [19]:
for hh_size, submodel in enumerate(model, 1):
    if hh_size > 3: 
        break 
        # the largest HH sizes have massive model reports that 
        # Excel doesn't handle well; review them in Jupyter 
        # instead if you are interested
    submodel.to_xlsx(
        result_dir/f"cdap_model_estimation_hhsize{hh_size}.xlsx", 
        data_statistics=False,
    )

# Next Steps

The final step is to either manually or automatically copy the `cdap_coefficients.csv_revised.csv` file to the configs folder, rename it to `cdap_coefficients.csv.csv`, and run ActivitySim in simulation mode.

In [20]:
pd.read_csv(result_dir/"cdap_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_UNAVAILABLE,-999.000000,T
1,coef_full_time_worker_asc_M,2.126748,F
2,coef_full_time_worker_asc_N,0.650847,F
3,coef_part_time_worker_asc_M,-0.902594,F
4,coef_part_time_worker_asc_N,0.677144,F
...,...,...,...
161,coef_N_xxxx,-1.346000,F
162,coef_H_xxxx,-3.733000,F
163,coef_M_xxxxx,0.103269,F
164,coef_N_xxxxx,-0.046852,F


# Modify Spec

Here, we will demonstrate the process of re-estimating the CDAP model with a modified
SPEC file.  This does *not* require re-running ActivitySim, it just requires
changing the SPEC file and re-running the Larch estimation only.

The `backup` command we ran earlier made a backup copy of the
original spec file in the EDB directory.
This was not strictly necessary, but since we're about to modify it and
we may want undo our changes, it can be handy to keep a copy of the
original spec file around. Since we already have a backup copy, we'll make some 
changes directly in the SPEC file.  As an example here, we're going
to re-write the household income section of the file, to change the piecewise 
linear utility from 3 segments to 4.  We'll move the breakpoints and rename some
coefficients to accomodate the change.  As above, for this demo we are editing 
the SPEC file using Python code to make the changes, but a user does not need
to change the file using Python; any CSV editor (e.g. Excel) can be used. 

In [21]:
with open(data.edb_directory / "cdap_INDIV_AND_HHSIZE1_SPEC.csv") as f:
    raw_spec = f.read()

repl_lines = """Full-time worker interaction with income less than $20k,(ptype == 1) & (income_in_thousands < 20),,,coef_full_time_worker_interaction_with_income_less_than_20k_H
"""

new_lines = """Full-time worker interaction with income less than $35k,(ptype == 1) & (income_in_thousands < 35),,,coef_full_time_worker_interaction_with_income_less_than_35k_H
Full-time worker interaction with income between $35k and $100k,(ptype == 1) & (income_in_thousands >= 35) & (income_in_thousands <= 100),,,coef_full_time_worker_interaction_with_income_between_35k_and_100k_H
"""

raw_spec = raw_spec.replace(repl_lines, new_lines)

with open(data.edb_directory / "cdap_INDIV_AND_HHSIZE1_SPEC.csv", "w") as f:
    f.write(raw_spec)

Unlike other components, the CDAP estimation process is built fundamentally against 
a re-evaluation of utility functions, instead of using pre-computed values in the
EDB. Thus, the user does not need to worry about the "Label" column of the utility
specification file (which in this example does not even exist.)  

Estimation of the revised model can be completed using the same process as the 
original estimation.

In [22]:
model2, data2 = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

Reading output-est-mode/estimation_data_bundle/cdap/cdap_model_settings.yaml
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimation-data/activitysim-prototype-mtc-extended/output-est-mode/final_households.csv
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimation-data/activitysim-prototype-mtc-extended/output-est-mode/final_persons.csv
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimation-data/activitysim-prototype-mtc-extended/output-est-mode/estimation_data_bundle/cdap/cdap_coefficients.csv
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimation-data/activitysim-prototype-mtc-extended/output-est-mode/estimation_data_bundle/cdap/cdap_interaction_coefficients.csv
Reading /Users/jpn/Git/est-mode/activitysim/activitysim/examples/example_estimation/notebooks/test-estimatio

In [23]:
r2 = model2.estimate(method='SLSQP', options={'maxiter':1000})

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
-999.0,-999.000000,-999.000000,-999.000000,-999.0,-999.0,0.0,1
coef_H_11,1.455012,1.455012,1.626000,-inf,inf,0.0,0
coef_H_12,0.949608,0.949608,0.740700,-inf,inf,0.0,0
coef_H_124_122_144,0.367496,0.367496,0.957300,-inf,inf,0.0,0
coef_H_126_146,-0.009021,-0.009021,0.293900,-inf,inf,0.0,0
...,...,...,...,...,...,...,...
coef_retired_interaction_with_more_cars_than_workers_M,2.992000,2.992000,2.992000,-inf,inf,0.0,0
coef_retired_interaction_with_more_cars_than_workers_N,1.012806,1.012806,1.056000,-inf,inf,0.0,0
coef_retired_interaction_with_peak_accessibility_to_all_employment_M,0.279200,0.279200,0.279200,-inf,inf,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/optimization.py:338: UserWarning: SLSQP may not play nicely with unbounded parameters
if you get poor results, consider setting global bounds with model.set_cap()
  warnings.warn(  # infinite bounds #  )


We can easily review the parameter estimates from the original and
revised models side by side to see what changed.

In [24]:
with pd.option_context('display.max_rows', 999):
    display(pd.concat({
        "model": model.estimation_statistics_raw(),
        "model2": model2.estimation_statistics_raw(),
    }, axis=1).fillna(""))

model        model2
Number of Cases                    Aggregate  20000.000000  20000.000000
Log Likelihood at Convergence      Aggregate -28223.515541 -28225.107151
                                   Per Case      -1.411176     -1.411255
Log Likelihood at Null Parameters  Aggregate -51107.531825 -51107.531825
                                   Per Case      -2.555377     -2.555377
Rho Squared w.r.t. Null Parameters Aggregate      0.447762      0.447731

In [25]:
with pd.option_context('display.max_rows', 999):
    display(pd.concat({
        "model": model.parameter_summary().data,
        "model2": model2.parameter_summary().data,
    }, axis=1).fillna(""))

model             \
                                                       Value    Std Err   
Parameter                                                                 
-999.0                                                 -999.       0.00   
coef_H_11                                               1.45     0.0614   
coef_H_12                                              0.955      0.132   
coef_H_124_122_144                                     0.354      0.153   
coef_H_126_146                                       -0.0134      0.336   
coef_H_13                                               1.20      0.130   
coef_H_14                                              0.998     0.0903   
coef_H_15                                               1.18     0.0922   
coef_H_16                                               1.85      0.102   
coef_H_17                                               1.34     0.0557   
coef_H_18                                              0.803      0.109   
coef_H_22                                               1.66      0.215   
coef_H_222_224_244                                    0.0747      0.225   
coef_H_226_246_446                                    0.0745      0.683   
coef_H_23                                               1.68      0.209   
coef_H_24                                               1.20      0.165   
coef_H_25                                            -0.0811      0.278   
coef_H_26                                               2.83      0.239   
coef_H_266_466                                          1.24      0.650   
coef_H_27                                              0.803      0.158   
coef_H_28                                              0.941      0.187   
coef_H_33                                              0.359      0.448   
coef_H_34                                               1.54      0.131   
coef_H_35                                              0.211      0.389   
coef_H_36                                               1.55   2.55e-13   
coef_H_37                                               1.55         NA   
coef_H_38                                               1.34         NA   
coef_H_44                                               1.09     0.0884   
coef_H_45                                               1.13      0.139   
coef_H_46                                              0.524   9.04e-13   
coef_H_47                                              0.811   4.46e-13   
coef_H_48                                               1.17   2.26e-13   
coef_H_55                                               1.22      0.166   
coef_H_56_57_58                                        0.863   1.52e-13   
coef_H_66                                               1.36      0.299   
coef_H_67                                               1.29      0.385   
coef_H_68                                               1.39      0.755   
coef_H_77                                               2.43     0.0802   
coef_H_78                                               1.21      0.157   
coef_H_88                                              0.944      0.218   
coef_H_xxxxx                                           -3.46      0.269   
coef_M_11                                            0.00775     0.0439   
coef_M_111                                             0.158     0.0551   
coef_M_112_114                                         0.297     0.0900   
coef_M_12                                             -0.109     0.0595   
coef_M_13                                              0.195     0.0448   
coef_M_16                                              0.158     0.0607   
coef_M_17                                             0.0366     0.0286   
coef_M_18                                              0.373     0.0438   
coef_M_22                                              0.964     0.0932   
coef_M_23                                              0.107     0.0905   
coef_